In [1]:
import numpy as np
from simple.ohlc import renko
from simple.plotly import chartFigure
from simple.funcs import symlog
from simple.backtest import npBacktestIOC

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

95820

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]
Buy[:5]

rec.array([('2022-05-08T00:01:00.335000', 0, 2636.,  6.),
           ('2022-05-08T00:01:00.335000', 1, 2636.,  4.),
           ('2022-05-08T00:01:00.668000', 2, 2636., 96.),
           ('2022-05-08T00:01:00.668000', 3, 2636.,  2.),
           ('2022-05-08T00:01:00.668000', 4, 2636.,  2.)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [5]:
len(T), len(S)

(95820, 99941)

In [6]:
C = renko(T, 15)
C[:5]

[(0, 17610, 2625.0, 2640.0),
 (17611, 18413, 2655.0, 2640.0),
 (18414, 18420, 2625.0, 2640.0),
 (18421, 18421, 2655.0, 2640.0),
 (18422, 52272, 2640.0, 2625.0)]

In [7]:
len(C)

64

In [8]:
delay = np.timedelta64(10, 'ms')

In [9]:
signal = 0
signalA = np.zeros(len(S.DT))

for c in C:
    color = 'green' if c[3] > c[2] else 'red'
    duration = T.DateTimeA[c[1]] - T.DateTimeA[c[0]]
    tm = T.DateTimeA[c[1]] + delay
    k = np.searchsorted(S.DT, tm)
    if duration < np.timedelta64(10000, 'μs'):
        if c[3] > c[2]:
            signal = 1
        elif c[3] < c[2]:
            signal = -1
    if k < len(signalA):
        signalA[k] = signal

    print(f'{str(c): <26}', T.DateTimeA[c[1]], tm, f'{duration: <16}', k, signal, color, sep='\t')

(0, 17610, 2625.0, 2640.0)	2022-05-08T00:10:12.393999	2022-05-08T00:10:12.403999	552058999 microseconds	15752	0	green
(17611, 18413, 2655.0, 2640.0)	2022-05-08T00:10:36.481000	2022-05-08T00:10:36.491000	24072001 microseconds	16434	0	red
(18414, 18420, 2625.0, 2640.0)	2022-05-08T00:10:36.481000	2022-05-08T00:10:36.491000	0 microseconds  	16434	1	green
(18421, 18421, 2655.0, 2640.0)	2022-05-08T00:10:36.496000	2022-05-08T00:10:36.506000	0 microseconds  	16435	-1	red
(18422, 52272, 2640.0, 2625.0)	2022-05-08T00:30:48.915000	2022-05-08T00:30:48.925000	1212399000 microseconds	50468	-1	red
(52273, 52353, 2610.0, 2625.0)	2022-05-08T00:30:48.921000	2022-05-08T00:30:48.931000	0 microseconds  	50469	1	green
(52354, 52376, 2640.0, 2625.0)	2022-05-08T00:30:49.108999	2022-05-08T00:30:49.118999	175999 microseconds	50474	1	red
(52377, 52381, 2610.0, 2625.0)	2022-05-08T00:30:49.116999	2022-05-08T00:30:49.126999	8000 microseconds	50475	1	green
(52382, 52382, 2640.0, 2625.0)	2022-05-08T00:30:49.135999	20

In [10]:
trades = npBacktestIOC(S.DT.astype(np.int64), S.A, S.B, signalA, 0)
trades

rec.array([(16435, '2022-05-08T00:10:36.523000', 2639.86010742, 2639.83007812, 16436, '2022-05-08T00:10:36.556000', 2639.85009766, 2639.85498047,  1.),
           (50469, '2022-05-08T00:30:48.959000', 2625.38989258, 2625.62988281, 50470, '2022-05-08T00:30:48.992000', 2625.54003906, 2625.51513672, -1.),
           (50475, '2022-05-08T00:30:49.155000', 2625.01000977, 2624.83007812, 50476, '2022-05-08T00:30:49.187999', 2624.62988281, 2624.81982422,  1.),
           (50820, '2022-05-08T00:31:00.856000', 2624.67993164, 2624.72998047, 50821, '2022-05-08T00:31:00.888000', 2624.97998047, 2624.875     , -1.),
           (50822, '2022-05-08T00:31:00.922000', 2625.2800293 , 2625.0300293 , 50823, '2022-05-08T00:31:00.953999', 2625.12988281, 2625.22998047,  1.),
           (50824, '2022-05-08T00:31:00.986999', 2625.37988281, 2625.26489258, 55715, '2022-05-08T00:33:53.917000', 2624.82006836, 2624.84008789,  1.),
           (56185, '2022-05-08T00:34:10.629000', 2625.05004883, 2625.05517578, 83417, '2

In [11]:
Long = trades[trades.Size > 0]
Short = trades[trades.Size < 0]

In [12]:
# timed chart
fig = chartFigure(rows=2,
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.2),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.2),
    Ask=dict(x=S.DT, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
    Bid=dict(x=S.DT, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines'),

    EnterLong=dict(x=Long.T0, y=Long.Price0, mode='markers',
                   marker=dict(symbol='triangle-up', size=12, line_color='darkgreen', line_width=1, color='green')),
    ExitLong=dict(x=Long.T1, y=Long.Price1, mode='markers',
                  marker=dict(symbol='x', size=10, line_color='darkgreen', line_width=1, color='green')),

    EnterShort=dict(x=Short.T0, y=Short.Price0, mode='markers',
                    marker=dict(symbol='triangle-down', size=12, line_color='darkred', line_width=1, color='red')),
    ExitShort=dict(x=Short.T1, y=Short.Price1, mode='markers',
                   marker=dict(symbol='x', size=10, line_color='darkred', line_width=1, color='red')),
    Signal=dict(x=S.DT, y=signalA, row=2, color='magenta', shape='hv')
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.2,
    …

In [13]:
fig.layout.shapes = [{
    'fillcolor': 'green' if c[3] > c[2] else 'red',
    'line': {'color': 'black', 'width': 2},
    'opacity': 0.15,
    'type': 'rect',
    'x0': str(T.DateTimeA[int(c[0])]),
    'x1': str(T.DateTimeA[int(c[1])]),
    'y0': c[2],
    'y1': c[3]
} for c in C]